## Generate LLM descriptions

In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata
import PIL.Image
import csv
import pandas as pd

api_key = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro')

In [ ]:
def get_all_images_by_subfolder(main_folder):
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')
    all_images_by_subfolder = []

    for root, dirs, files in os.walk(main_folder):
        if root == main_folder:
            continue  # Skip the main folder itself
        subfolder_images = []
        for file in files:
            if file.lower().endswith(image_extensions):
                subfolder_images.append(os.path.join(root, file))
        if subfolder_images:
            all_images_by_subfolder.append(subfolder_images)

    return all_images_by_subfolder

main_folder =  # Path to the folder containing subfolders with images
images_by_subfolder = get_all_images_by_subfolder(main_folder)
print(len(images_by_subfolder))

prompt = "Write a description for the given image sequence in a single paragraph, what is happening in this episode?"

found_bad = False

i = 0

for subfolder in images_by_subfolder:
  print("unsorted", subfolder)
  for file_name in subfolder:
    if "ipynb_checkpoints" in file_name:
      found_bad = True
      break

  if found_bad:
    found_bad = False
    continue

  i += 1

  current_image_paths = sorted(subfolder)
  print(current_image_paths)


  current_images = []

  for image in current_image_paths:
      img = PIL.Image.open(image)
      current_images.append(img)

  input = [prompt] + current_images

  print(prompt)
  print(current_images)
  print(input)

  response = model.generate_content(input)
  print(response.text)

  with open("llm_descriptions.csv", mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([current_image_paths[0].split('/')[-2], response.text])

## Load and sort human and LLM descriptions

In [ ]:
df = pd.read_csv("human_descriptions.csv")
df_sorted = df.sort_values("image_name", ignore_index=True)
df_llm = pd.read_csv("llm_descriptions.csv")
df_llm_sorted = df_llm.sort_values("image_name", ignore_index=True)

## Get aggregation summaries

In [ ]:
text_aggregation_prompt = """
You will get two versions of the same story. Please provide the following information:

1. Overlap: all the information that was in both versions.
2. Complementary information: all information that is only mentioned in one of the versions, but doesn't
conflict with anything in the other version.
3. Conflicting information: information that conflicts between the two versions.

Use exactly this format:

Overlap:
- overlap item 1
- overlap item 2
- ...

Complementary information description 1:
- item 1
- item 2
- ...

Complementary information description 2:
- item 1
- item 2
- ...

Conflicting information:
- item 1
- item 2
- ...


Here is the first version for you to use for your answer:
{first_description}

Here is the second version for you to use for your answer:
{second_description}
"""

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro')

for idx in df.index[:]:
  desc1 = df_sorted.loc[idx, 'gt_description']
  desc2 = df_llm_sorted.loc[idx, 'llm_description']

  prompt = text_aggregation_prompt.format(first_description=desc1, second_description=desc2)
  print(prompt)
  response = model.generate_content(prompt)
  with open("aggregation.csv", mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([df_sorted.loc[idx, 'image_name'], response.text])

## Create files with the two descriptions and the aggregation

#### full data

In [ ]:
df_desc1 = pd.read_csv("human_descriptions.csv")
df_desc1 = df_desc1.sort_values("image_name", ignore_index=True)
df_desc2 = pd.read_csv("llm_descriptions.csv")
df_desc2 = df_desc2.sort_values("image_name", ignore_index=True)
df_aggr = pd.read_csv("aggregation.csv")
df_aggr = df_aggr.sort_values("image_name", ignore_index=True)

#### partial data

In [ ]:
df_desc1_partial = pd.read_csv("description_partial.csv")
df_desc1_partial = df_desc1_partial.sort_values("image_name", ignore_index=True)
df_desc2_partial = pd.read_csv("llm_description_partial.csv")
df_desc2_partial = df_desc2_partial.sort_values("image_name", ignore_index=True)
df_aggr_partial = pd.read_csv("aggregation_partial.csv")
df_aggr_partial = df_aggr_partial.sort_values("image_name", ignore_index=True)

#### combining full and partial data per participant

In [ ]:
participants = [{"full": ["cmc_2.png", "cmc_40.png", "cmc_68.png"], "partial": ["cmc_34.png", "cmc_60.png"]},
               {"full": ["cmc_34.png", "cmc_60.png"], "partial": ["cmc_2.png", "cmc_40.png", "cmc_68.png"]},
               {"full": ["cmc_68.png", "cmc_40.png", "cmc_2.png"], "partial": ["cmc_60.png", "cmc_34.png"]},
               {"full": ["cmc_60.png", "cmc_34.png"], "partial": ["cmc_68.png", "cmc_40.png", "cmc_2.png"]}]

In [ ]:
merged_full = pd.merge(df_desc1, df_desc2, how="outer", on=["image_name"])
merged_full = pd.merge(merged_full, df_aggr, how="outer", on=["image_name"])
merged_partial = pd.merge(df_desc1_partial, df_desc2_partial, how="outer", on=["image_name"])
merged_partial = pd.merge(merged_partial, df_aggr_partial, how="outer", on=["image_name"])

In [ ]:
for i in range(4): # iterating over participants list
  full_samples = participants[i]["full"]
  partial_samples = participants[i]["partial"]

  df_full_temp = merged_full[merged_full["image_name"].isin(full_samples)]
  df_partial_temp = merged_partial[merged_partial["image_name"].isin(partial_samples)]

  df_participant = pd.concat([df_full_temp, df_partial_temp])
  df_participant = df_participant.drop("image_name", axis=1)

  df_participant['ID'] = df_participant.index

  df_participant = df_participant.rename(columns={"gt_description": "Text1", "llm_description": "Text2", "aggregation": "Summary"})

  cols = df_participant.columns.tolist()
  cols = cols[-1:] + cols[:-1]
  df_participant = df_participant[cols]

  df_participant = df_participant.sort_values("ID")

  df_participant.to_csv(f"participant_{i+1}_data.tsv", sep="\t", index=False) # save to file